## Preprocessing

In [24]:
import json, re

def load(path: str):
    with open(path, 'r') as f:
        return json.load(f)
    
def preprocess (line: str) -> str: 
    regStart = re.compile(r'^.* Lyrics\n')
    argStart = regStart.sub('', line)
    regEnd = re.compile(r'\d+Embed$')
    argEnd = regEnd.sub('', argStart)
    return argEnd
    
lines = load('./data/songs.json')
lines = [preprocess(line["row"]["text"]) for line in lines["rows"] if line["row"]["text"]]

lines

['The club isnt the best place to find a lover\nSo the bar is where I go\nMe and my friends at the table doing shots\nDrinking fast and then we talk slow\nAnd you come over and start up a conversation with just me\nAnd trust me Ill give it a chance now\nTake my hand, stop, put Van the Man on the jukebox\nAnd then we start to dance, and now Im singing like\nGirl, you know I want your love\nYour love was handmade for somebody like me\nCome on now, follow my lead\nI may be crazy, dont mind me\nSay, boy, lets not talk too much\nGrab on my waist and put that body on me\nCome on now, follow my lead\nCome, come on now, follow my lead\nIm in love with the shape of you\nWe push and pull like a magnet do\nAlthough my heart is falling too\nIm in love with your body\nAnd last night you were in my room\nAnd now my bed sheets smell like you\nEvery day discovering something brand new\nIm in love with your body\nOh, I, oh, I, oh, I, oh, I\nIm in love with your body\nOh, I, oh, I, oh, I, oh, I\nIm in l

## Dictionnary

In [27]:
dict = {}

for line in lines:
    words = re.split(r'\s+|\n+', line)
    bigrams = [words[i] + "+" + words[i+1] for i in range(len(words)-1)]

    for bigram in bigrams:
        if bigram in dict:
            dict[bigram] += 1
        else:
            dict[bigram] = 1

# Sort the dictionary by value in descending order
dict = sorted(dict.items(), key=lambda x: x[1], reverse=True)

dict

[('I+dont', 165),
 ('in+love', 131),
 ('in+the', 131),
 ('And+I', 116),
 ('on+the', 114),
 ('I+know', 111),
 ('Im+in', 89),
 ('in+my', 75),
 ('need+you', 75),
 ('to+the', 69),
 ('in+a', 68),
 ('I+love', 68),
 ('dont+wanna', 67),
 ('Ill+be', 66),
 ('love+with', 64),
 ('Ive+been', 64),
 ('I+was', 63),
 ('You+need', 63),
 ('I+wanna', 62),
 ('need+me,', 61),
 ('man,+I', 61),
 ('on+my', 60),
 ('and+I', 59),
 ('of+the', 57),
 ('me,+man,', 57),
 ('give+me', 55),
 ('that+I', 54),
 ('wanna+be', 53),
 ('with+the', 50),
 ('I+just', 50),
 ('I+can', 50),
 ('I+could', 50),
 ('dont+need', 50),
 ('you+You', 48),
 ('the+way', 47),
 ('to+be', 45),
 ('I+got', 45),
 ('for+a', 45),
 ('You+know', 44),
 ('like+a', 43),
 ('with+you', 43),
 ('your+body', 42),
 ('if+you', 42),
 ('know+that', 41),
 ('with+your', 40),
 ('in+your', 40),
 ('with+a', 39),
 ('I+will', 38),
 ('when+I', 38),
 ('out+of', 38),
 ('back+to', 37),
 ('with+me', 37),
 ('wanna+know', 37),
 ('you+know', 36),
 ('my+baby,', 36),
 ('be+your', 36),